In [ ]:
!pip install ultralytics opencv-python opencv-python-headless scikit-image
!pip install filterpy lap

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
video_folder = '/content/drive/MyDrive/vedios'

# Iterate over all files in the folder
for filename in os.listdir(video_folder):
  print(filename)

In [ ]:
# from deep_sort_realtime.deepsort_tracker import DeepSort
!pip install deep_sort_realtime

In [ ]:
import cv2
import torch
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# Load YOLOv8 model (pre-trained)
model = YOLO('yolov8l.pt')  # you can use a larger model like yolov8l.pt if needed

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=50, n_init=2, nn_budget=100)

# Define labels for child and adult
child_label = "child"
adult_label = "therapist"

# Function to detect and track persons
def detect_and_track(video_path, output_path):
    # Open video capture
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define codec and create VideoWriter object
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detect persons in the frame
        results = model(frame)

        # YOLOv8 provides the detection results, loop through each detection
        detections = []
        for result in results:
            for bbox in result.boxes:
                # Bounding box coordinates and confidence score
                x1, y1, x2, y2 = bbox.xyxy[0].tolist()
                conf = bbox.conf.item()
                label_idx = int(bbox.cls.item())  # Get class index

                # Filtering out low confidence detections and non-person objects
                if label_idx == 0 and conf > 0.7:  # Label 0 corresponds to "person"
                    # Append bounding box coordinates, confidence, and class label (optional)
                    detections.append([[x1, y1, x2, y2], conf])

        # Update tracker with detections
        tracked_objects = tracker.update_tracks(detections, frame=frame)

        # Draw bounding boxes and labels
        for track in tracked_objects:
            bbox = track.to_tlbr()  # Bounding box
            track_id = track.track_id  # Unique track ID

            # Choose label (child or adult) based on some criteria (e.g., bounding box size)
            label = child_label if bbox[3] - bbox[1] < 240 else adult_label
            if label==child_label:
              # Draw bounding box
              cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (255, 0, 0), 2)
              # Put unique ID and label on the frame
              cv2.putText(frame, f"{label} {track_id}", (int(bbox[0]), int(bbox[1]) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            else:

              # Draw bounding box
              cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 255, 0), 2)
              # Put unique ID and label on the frame
              cv2.putText(frame, f"{label} {track_id}", (int(bbox[0]), int(bbox[1]) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Write the frame with predictions
        out.write(frame)

        # Optionally display frame in a window (for debugging)
        # cv2.imshow("Frame", frame)
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break

    # Release everything
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Run detection and tracking on the test video
# c=0
video_path = '/content/drive/MyDrive/vedios/input2.mp4'
output_path = '/content/output2.mp4'
detect_and_track(video_path, output_path)

# for filename in os.listdir(video_folder):
#   # print(filename)
#   video_path = '/content/drive/MyDrive/vedios/'+filename
#   output_path = '/content/output/output'+str(c)+'.mp4'
#   c+=1
#   detect_and_track(video_path, output_path)


In [ ]:
import cv2
import torch
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# Load YOLOv8 model (pre-trained)
model = YOLO('yolov8l.pt')  # you can use a larger model like yolov8l.pt if needed

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=50, n_init=2, nn_budget=100)

# Define labels for child and adult
child_label = "child"
adult_label = "adult"

# Function to detect and track persons
def detect_and_track(video_path, output_path):
    # Open video capture
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define codec and create VideoWriter object
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detect persons in the frame
        results = model(frame)

        # YOLOv8 provides the detection results, loop through each detection
        detections = []
        for result in results:
            for bbox in result.boxes:
                # Bounding box coordinates and confidence score
                x1, y1, x2, y2 = bbox.xyxy[0].tolist()
                conf = bbox.conf.item()
                label_idx = int(bbox.cls.item())  # Get class index

                # Filtering out low confidence detections and non-person objects
                if label_idx == 0 and conf > 0.7:  # Label 0 corresponds to "person"
                    # Append bounding box coordinates, confidence, and class label (optional)
                    detections.append([[x1, y1, x2, y2], conf])

        # Update tracker with detections
        tracked_objects = tracker.update_tracks(detections, frame=frame)

        # Draw bounding boxes and labels
        for track in tracked_objects:
            bbox = track.to_tlbr()  # Bounding box
            track_id = track.track_id  # Unique track ID

            # Choose label (child or adult) based on some criteria (e.g., bounding box size)
            label = child_label if bbox[3] - bbox[1] < 240 else adult_label
            if label==child_label:
              # Draw bounding box
              cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (255, 0, 0), 2)
              # Put unique ID and label on the frame
              cv2.putText(frame, f"{label} {track_id}", (int(bbox[0]), int(bbox[1]) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            else:

              # Draw bounding box
              cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 255, 0), 2)
              # Put unique ID and label on the frame
              cv2.putText(frame, f"{label} {track_id}", (int(bbox[0]), int(bbox[1]) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Write the frame with predictions
        out.write(frame)

        # Optionally display frame in a window (for debugging)
        # cv2.imshow("Frame", frame)
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break

    # Release everything
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Run detection and tracking on the test video
# c=0
video_path = '/content/drive/MyDrive/vedios/input3.mp4'
output_path = '/content/output3.mp4'
detect_and_track(video_path, output_path)

# for filename in os.listdir(video_folder):
#   # print(filename)
#   video_path = '/content/drive/MyDrive/vedios/'+filename
#   output_path = '/content/output/output'+str(c)+'.mp4'
#   c+=1
#   detect_and_track(video_path, output_path)
